# **Récolte des données** 

### Modification à faire :

- Retravailler les cellules markdown et les commentaires Python
- Restructurer les paramètres

### Importation des librairies utiles

In [210]:
import requests
import json
import pandas as pd
import overpy
import geopy.distance as gd

### Création du dictionnaire des Localisation avec leur URL respectives

In [211]:
#Localisation des données
dict_association_localisation = {"AGGLOMÉRATION LE COTENTIN":'https://vigilo.velorution-cherbourg.fr/get_issues.php?',
                                 "AGGLOMÉRATION TROYENNE":'https://vigilo.troyesenselle.fr/get_issues.php?', 
                                 "AIX MARSEILLE PROVENCE METROPOLE":'https://vigilo.velosenville.org/get_issues.php?',
                                 "BASSIN D'ARCACHON":'https://vigilo.velobassin.org/get_issues.php?',
                                 "BEAUVAISIS":'https://vigilo.vellovaque.fr/get_issues.php?',
                                 "BORDEAUX MÉTROPOLE":'https://vigilo.cycles-manivelles.org/get_issues.php?',
                                 "BREST MÉTROPOLE":'https://vigilo.bapav.org/get_issues.php?',
                                 "EVREUX":'https://vigilo.cube27.org/get_issues.php?',
                                 "GRAND MONTPELLIER":'https://vigilo.jesuisundesdeux.org/get_issues.php?',
                                 "LA ROCHELLE":'https://vigilo.-lr.corolo.fr/get_issues.php?',
                                 "LORIENT AGGLO":'https://vigilo.claav.fr/get_issues.php?',
                                 "MONS":'https://vigilo.geckonline.be/get_issues.php?',
                                 "NANCY AGGLOMÉRATION":'https://vigilo.rigelk.eu/get_issues.php?',
                                 "NANTES MÉTROPOLE":'https://vigilo.placeauvelo-nantes.fr/get_issues.php?',
                                 "NORD-ISÈRE (APIE)":'https://vigilo.apie-asso.net/get_issues.php?',
                                 "PARIS SACLAY":'https://api.vigilo-parissaclay.mdb-idf.org/get_issues.php?',
                                 "PAYS DE CHÂTEAUGIRON":'https://pouliquen.net/vigilo/get_issues.php?',
                                 "PÉRIGUEUX AGGLO":'https://vigilo.velorution-perigourdine.fr/get_issues.php?',
                                 "SAINT MARCELLIN VERCORS ISÈRE COMMUNAUTÉ":'https://vigilo.roulavelo.org/get_issues.php?',
                                 "SAINT-BRIEUC ARMOR AGGLOMÉRATION":'https://api-vigilo.velo-utile.fr/get_issues.php?',
                                 "STRASBOURG":'https://www.strasbourgavelo.eu/get_issues.php?',
                                 "SUD OISE":'https://vigilo.au5v.fr//get_issues.php?',
                                 "THAU":'https://prod-sete.vigilo.velocite-montpellier.fr/get_issues.php?',
                                 "VANNES":'https://vigilo.kaz.bzh/get_issues.php?'
                                }

df_localisation = pd.DataFrame(dict_association_localisation.keys(), columns=["Localisation"])
df_localisation.set_index("Localisation", inplace=True)
df_localisation

""
Localisation
AGGLOMÉRATION LE COTENTIN
AGGLOMÉRATION TROYENNE
AIX MARSEILLE PROVENCE METROPOLE
BASSIN D'ARCACHON
BEAUVAISIS
BORDEAUX MÉTROPOLE
BREST MÉTROPOLE
EVREUX
GRAND MONTPELLIER


### Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [212]:
# URL de l'API pour récupérer la liste des catégories
url_categories = 'https://vigilo-bf7f2.firebaseio.com/categorieslist.json'
resp = requests.get(url_categories)
resp.raise_for_status()

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {str(cat["catid"]): cat["catname"] for cat in resp.json()}

# Créer un dataframe à partir du dictionnaire
df_categories = pd.DataFrame(categories_dict.items(), columns=["id", "Catégories"]).set_index("id")

# Afficher le dataframe
df_categories

,Catégories
id,
8,Absence d'aménagement
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
2,Véhicule ou objet gênant
9,"Accident, chute, incident"
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
10,Vol ou dégradation de vélo


### Liste des POI

In [213]:
# Créer un dictionnaire des points d'intérêt et leur identifiant OSM (ex: pour les parkings à vélo : "amenity:bicycle_parking")

poi_dict = {
    "Écoles": "amenity=school",
    "Hôpitaux": "amenity=hospital",
    "Parkings à vélos": "amenity=bicycle_parking",
    "Stations de vélos en libre-service": "amenity=bicycle_rental",
    "Passages piétons": "highway=crossing",
    "Feux piétons": "highway=traffic_signals",
    "Places": "highway=pedestrian",
    "Rampes d'accès pour handicapés": "amenity=wheelchair_ramp",
    "Sièges pour handicapés": "amenity=wheelchair_seating",
    "Parkings pour handicapés": "amenity=parking_space",
    "Parcs": "leisure=park",
    "Jardins": "leisure=garden",
    "Terrains de jeux": "leisure=playground",
    "Toilettes publiques": "amenity=toilets",
    "Fontaines à eau": "amenity=drinking_water",
    "Marchés": "amenity=marketplace",
    "Supermarchés": "shop=supermarket",
    "Boulangeries": "shop=bakery",
    "Pharmacies": "amenity=pharmacy",
    "Bibliothèques": "amenity=library",
    "Bureaux de poste": "amenity=post_office",
    "Stations de recharge pour véhicules électriques": "amenity=charging_station",
    "Cliniques": "amenity=clinic",
    "Dentistes": "amenity=dentist",
    "Vétérinaires": "amenity=veterinary",
    "Points de vue": "tourism=viewpoint",
    "Hôtels": "tourism=hotel",
    "Aires de pique-nique": "leisure=picnic_table",
    "Écoles maternelles": "amenity=kindergarten",
    "Écoles primaires": "amenity=primary_school",
    "Écoles secondaires": "amenity=secondary_school"
}

# Créer un dataframe à partir du dictionnaire
df_poi = pd.DataFrame(poi_dict.items(), columns=["Points d'intérêt", "Identifiant OSM"])

# Changer l'index
df_poi = df_poi.set_index("Identifiant OSM")

# Afficher le dataframe
df_poi

,Points d'intérêt
Identifiant OSM,
amenity=school,Écoles
amenity=hospital,Hôpitaux
amenity=bicycle_parking,Parkings à vélos
amenity=bicycle_rental,Stations de vélos en libre-service
highway=crossing,Passages piétons
highway=traffic_signals,Feux piétons
highway=pedestrian,Places
amenity=wheelchair_ramp,Rampes d'accès pour handicapés
amenity=wheelchair_seating,Sièges pour handicapés


### Initialisation des paramètres

- Choississez une localisation dans la liste
- Choississez une catégorie et entrez son identifiant (id)
- Choississez un ou plusieurs points d'intérêts
- Définissez la boîte de recherche, pour cela, choississez sur 'site' un point au centre de l'endroit qui vous intéresse (centre de la ville) et entrez la latitude et la longitude de ce point dans central_lat et central_lon respectivement

In [214]:
# Paramètres de recherche

# Filtre de recherche par localisation, exemple : localisation = "BREST MÉTROPOLE"
location_filter = "BREST MÉTROPOLE"

# FIltre de recherche par date, exemple : date_filter = "2021-01-01" dans le format "YYYY-MM-DD hh:mm:ss"
date_debut = "2000-01-01 00:00:00"
date_fin = "2024-12-31 23:59:59"

# Filtre des catégories à afficher, exemple : category_filter = "8" donne la catégorie Absence d'aménagement
category_filter = None

# Filtre des points d'intérêt à afficher, exemple : poi_filter = "Parking à vélo" donne le point d'intérêt Parking à vélo
poi_names = ["Écoles", "Hôpitaux", "Passages piétons", "Parkings pour handicapés", "Parcs", "Jardins", "Terrains de jeux", "Fontaines à eau", "Marchés", "Supermarchés", "Boulangeries", "Pharmacies", "Stations de recharge pour véhicules électriques", "Dentistes", "Vétérinaires", "Points de vue", "Hôtels", "Aires de pique-nique", "Écoles maternelles", "Écoles primaires"]

# Boîte de recherche pour les points d'intérêt
central_lat = 48.390177
central_lon = -4.496499
delta = 0.02

bbox = [
    central_lat - delta,  # south latitude
    central_lon - delta,  # west longitude
    central_lat + delta,  # north latitude
    central_lon + delta   # east longitude
]

### Création du dataframe des données

In [215]:
def data_collection(location_filter, category_filter, date_debut=None, date_fin=None):
    try:
        # Construct the URL for the API call
        url = dict_association_localisation[location_filter] + ('c=' + category_filter if category_filter is not None else '')
        resp = requests.get(url)
        resp.raise_for_status()  # Raises an exception for HTTP errors

        # Load data into DataFrame
        df = pd.DataFrame(resp.json())

        # Ensure 'time' column is treated as integer for Unix timestamp conversion
        # Explicitly cast to integer before conversion
        df['time'] = pd.to_datetime(df['time'].astype(int), unit='s')

        # Convert datetime back to Unix timestamp for comparison (if filtering is needed)
        if date_debut is not None and date_fin is not None:
            date_debut_dt = pd.to_datetime(date_debut).timestamp()
            date_fin_dt = pd.to_datetime(date_fin).timestamp()
            df = df[(df['time'].astype('int64') // 10**9 >= date_debut_dt) & 
                    (df['time'].astype('int64') // 10**9 <= date_fin_dt)]

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON or date conversion issue: {e}")
        return None
df = data_collection(location_filter, category_filter, date_debut, date_fin)

### Aperçu des données récoltées

In [216]:
df.head()

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
0,63457DB8,48.41369998623073,-4.482733014638274,"Boulevard de l\'Europe, Brest",Arbres cachant la signalisation,"Ici, les voitures roulent allègrement à vive a...",2024-04-01 14:01:00,0,0,4,1,Brest
1,DD7230F6,48.41161342100428,-4.489712102838329,"Rue Honoré de Balzac, Brest",Stationnement sur trottoir VS pietons,,2023-12-22 06:46:00,0,0,2,1,Brest
2,7FA00E08,48.4123682974901,-4.4859784307144155,"Rue Étienne Hubac, Brest",Stationnement sur trottoir VS pietons,Une rue plus qu\'assez large pour stationner d...,2023-12-22 06:42:00,0,0,2,1,Brest
3,AFCA2E97,48.39148377417681,-4.469872824847698,"Pierre Semard , Brest",stationnement sur la piste cyclable,,2023-12-21 18:45:00,0,0,2,1,Brest
4,B7F5BC01,48.39149290224677,-4.469862096011639,"Pierre Semard , Brest",Stationnement sur piste cyclabe,,2023-12-20 18:43:00,0,0,2,1,Brest


### Récupération des données OpenStreetMap autour de la région cherchée

In [217]:
def fetch_poi_data(bbox, poi_names, poi_dict):
    # Prepare the Overpass query part for multiple POI tags
    query_parts = []
    for poi_name in poi_names:
        poi_tag = poi_dict.get(poi_name)
        if poi_tag is None:
            raise ValueError(f"Le point d'intérêt '{poi_name}' n'est pas reconnu.")
        # Format tag correctly for the Overpass query
        formatted_poi_tag = poi_tag.replace("=", '="') + '"'
        query_part = f"""
            node[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
            way[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
            rel[{formatted_poi_tag}](bbox:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
        """
        query_parts.append(query_part)
    
    # Combine all query parts
    full_query = f"""
    [out:json];
    (
        {''.join(query_parts)}
    );
    out center;
    """
    
    # Define the Overpass API URL
    url = "http://overpass-api.de/api/interpreter"

    # Send the request to the Overpass API
    response = requests.get(url, params={'data': full_query})
    response.raise_for_status()  # Raises an HTTPError for bad responses
    return response.json()

def process_poi_data(json_data):
    # Initialize a dictionary to hold all POIs
    poi_dict = {}
    
    # Go through each element in the JSON data
    for element in json_data['elements']:
        # Check if the element is a node and has the required 'tags'
        if element['type'] == 'node' and 'tags' in element:
            # Check if the latitude and longitude are within the bounding box
            if bbox[0] <= element['lat'] <= bbox[2] and bbox[1] <= element['lon'] <= bbox[3]:
                poi_info = {
                    'name': element['tags'].get('name', 'Unknown'),
                    'latitude': element.get('lat', None),
                    'longitude': element.get('lon', None),
                    }
                
                # Add this POI's information to the main dictionary
                poi_dict[element['id']] = poi_info
            
    return poi_dict

pois_json = fetch_poi_data(bbox, poi_names, poi_dict)
pois = process_poi_data(pois_json)
df_poi = pd.DataFrame.from_dict(pois, orient='index')
df_poi.head()

,name,latitude,longitude
34139698,Unknown,48.390844,-4.508180
34139781,Unknown,48.391240,-4.505862
34139940,Unknown,48.396253,-4.484091
34139955,Unknown,48.394030,-4.494016
34588174,Unknown,48.383483,-4.493343


### Combinaison des données Vigilo et OpenStreetMap